In [34]:
import numpy as np
import pandas as pd
from itertools import combinations

In [35]:
all_dups = pd.read_csv('data/all_dups/census_dpl.tsv', sep='\t')
all_dups



,id1,id2
0,1,2
1,1,3
2,1,4
3,2,3
4,2,4
...,...,...
371,837,447
372,838,441
373,839,442
374,840,448


In [36]:
found_dups = pd.read_csv('data/detected_dups/census_found_dups.csv')
found_dups

,dedup_id,first_name,last_name,middle_name,street_address,zip_code,ID,dropped
0,1,NaN,GOINS,R,ELDORADO,115.0,128,NO
1,1,NaN,GOINS,NaN,ELDORADO,115.0,129,YES
2,2,NaN,GOWANS,D,MALIBU,6520.0,130,NO
3,2,NaN,GOWANS,M,MALIBU,6520.0,131,YES
4,3,NaN,MATKIN,L,MALIBU,6514.0,151,NO
...,...,...,...,...,...,...,...,...
543,272,XANTHA,PAIGE,NaN,BASSWOOD,4830.0,519,YES
544,273,ZAKIYA,1RUMOR,A,HWY 54,3820.0,178,NO
545,273,ZAKAYA,KUMOR,C,HWY 54,3820.0,650,YES
546,274,ZIONARA,ERVINE,T,MARTIN LUTHER K,601.0,223,NO


In [84]:
found_dups = pd.read_csv('data/detected_dups/census_found_dups.csv')

def compare_duplicates(found_dups, true_dups):
    """
    Compare found duplicates against true duplicates and calculate statistics.

    :param found_dups: DataFrame containing the found duplicates with dedup_id and IDs.
    :param true_dups: DataFrame containing the true duplicates as id1 and id2.
    :return: Dictionary with calculated statistics.
    """
    # Prepare true duplicates as a set of tuples for quick comparison
    true_duplicates_set = set(
        [tuple(sorted(pair)) for pair in zip(true_dups['id1'], true_dups['id2'])]
    )

    # Extract found duplicates as a set of tuples (check for bidirectional matching)
    found_duplicates_set = set()
    for dedup_id, group in found_dups.groupby('dedup_id'):
        ids = group['ID'].unique()
        found_duplicates_set.update([tuple(sorted(pair)) for pair in combinations(ids, 2)])

    # Calculate statistics
    true_positives = found_duplicates_set.intersection(true_duplicates_set)
    false_positives = found_duplicates_set.difference(true_duplicates_set)
    false_negatives = true_duplicates_set.difference(found_duplicates_set)
    true_negatives = found_duplicates_set.difference(true_duplicates_set)

    num_true_duplicates = len(true_duplicates_set)
    num_detected_duplicates = len(true_positives)
    detection_percentage = (num_detected_duplicates / num_true_duplicates * 100) if num_true_duplicates > 0 else 0

    stats = {
        "true_positives": len(true_positives),
        "false_positives": len(false_positives),
        "false_negatives": len(false_negatives),
        "true_negatives": len(true_negatives),
        "num_true_duplicates": num_true_duplicates,
        "num_detected_duplicates": num_detected_duplicates,
        "detection_percentage": detection_percentage,
    }

    return stats


# Run the comparison
statistics = compare_duplicates(found_dups, all_dups)

# Display the results
statistics


{'true_positives': 65,
 'false_positives': 58,
 'false_negatives': 311,
 'true_negatives': 58,
 'num_true_duplicates': 376,
 'num_detected_duplicates': 65,
 'detection_percentage': 17.287234042553195}